# Generate new Images using StyleGAN

## Import necessary libraries 

Comment if is a local environment and execute this command in Anaconda terminal 

In [1]:
pip install python-resize-image

In [2]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import cv2
from PIL import Image
from resizeimage import resizeimage

#Comment if is a local environment 
from google.colab import drive
drive.mount('/content/drive')

#StyleGAN supports only version 1.x of Tensorflow 
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TensorFlow 1.x selected.
1.15.2


if you are using Google Colab, it is recommended to check if the notebook is the model is executed in a GPU 

In [3]:
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('Encontrada GPU: {}'.format(device_name))

Encontrada GPU: /device:GPU:0


## Clone StyleGAN github repository

In [4]:
!git clone https://github.com/NVlabs/stylegan.git
!ls /content/stylegan/

Cloning into 'stylegan'...
remote: Enumerating objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.
config.py	     LICENSE.txt	    run_metrics.py
dataset_tool.py      metrics		    stylegan-teaser.png
dnnlib		     pretrained_example.py  training
generate_figures.py  README.md		    train.py


In [5]:
sys.path.insert(0, "/content/stylegan")
import dnnlib

## Analyzing real images sizes 

In [6]:
data_dir= "/content/drive/MyDrive/Colab Notebooks/pokemon"

In [7]:
dim1=[]
dim2=[]

for image_name in os.listdir(data_dir+"/pokemon"):
    image=plt.imread(data_dir+"/pokemon/"+image_name)
    d1,d2,q= image.shape
    dim1.append(d1)
    dim2.append(d2)

In [8]:
np.mean(dim1)

64.0

In [9]:
np.mean(dim2)

64.0

In [10]:
# Change the route to a new one if you dont want to overwrite the original images 
for filename in tqdm(os.listdir(data_dir+"/pokemon"),desc ='reading images ...'):
  image = Image.open(data_dir+"/pokemon/"+filename).convert('RGB')
  image = image.resize((64,64))
  image.save(data_dir+"/pokemon/"+filename, image.format)

reading images ...: 100%|██████████| 819/819 [00:09<00:00, 82.24it/s]


## Convert the custom images to tfrecords

In [11]:
! python /content/stylegan/dataset_tool.py create_from_images /content/stylegan/datasets/custom-dataset '/content/drive/MyDrive/Colab Notebooks/pokemon/pokemon'




Loading images from "/content/drive/MyDrive/Colab Notebooks/pokemon/pokemon"
Creating dataset "/content/stylegan/datasets/custom-dataset"



/content/stylegan/dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 819 images.


## Begin Training the SGAN Model 

Before beginning training the model, make sure you make these two changes:


*   Go to the stylegan folder and open the train.py file, in line no 37 add the next route: "/content/stylegan/datasets/custom-dataset"
*   Go to the path under stylegan/metrics and open python file frechet_inception_distance.py. In line number 29 add the next link: 'https://nvlabs-fi-cdn.nvidia.com/stylegan/networks/metrics/inception_v3_features.pkl'



NOTE!: Using the GPU in Colab is time-limited, specially in training tasks. If you want to use until the training is complete, you need to execute the following Js function in the console of the opened notebook 

In [ ]:
function ClickConnect(){
console.log("Working");
document.querySelector("colab-connect-button").shadowRoot.getElementById("connect").click();
}
setInterval(ClickConnect,60000)

In [ ]:
! python /content/stylegan/train.py 




Creating the run dir: results/00002-sgan-/content/stylegan/datasets/custom-dataset-1gpu
Copying files to the run dir
dnnlib: Running training.training_loop.training_loop() on localhost...



Streaming data using training.dataset.TFRecordDataset...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Dataset shape = [3, 64, 64]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...







G                           Params    OutputShape       Weight

## Use pre-trained weights to generate images from the custom dataset

In [25]:
import pickle
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
import PIL.Image

In [ ]:
def main():
  # Initialize TensorFlow.
  tflib.init_tf()
  url = '/content/network-snapshot-003685.pkl'

  with open(url,'rb') as f :

    _G, _D, Gs = pickle.load(f)

    # _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
    # _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
    # Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

    # Print network details.
    Gs.print_layers()

    # Pick latent vector.
    rnd = np.random.RandomState()
    latents = rnd.randn(1, Gs.input_shape[1])

    # Generate image.

    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    # Save image.

    os.makedirs(config.result_dir, exist_ok=True)

    png_filename = os.path.join(config.result_dir, f’/content/example1.png’)

    PIL.Image.fromarray(images[0], ‘RGB’).save(png_filename)

#if __name__ == "__main__":
main()

# On running this code , output image example1.png will be created under /content. The output quality will be based on the network_snapshot.pkl we use